# REGO3 - Notebook 03 : Feature Engineering

Ce notebook crée de nouvelles features avancées à partir des variables existantes.

## Types de features
1. **Ratios** : Division de deux variables (ex: surface par étage)
2. **Interactions** : Multiplication de deux variables
3. **Temporelles** : Âge du bâtiment, ancienneté
4. **Agrégations** : Moyennes par quartier/type (FIT/TRANSFORM)
5. **Polynomiales** : Puissances, racines

## Inputs
* `train_processed.csv` : Données train du notebook 02
* `test_processed.csv` : Données test du notebook 02

## Outputs
* `train_with_features.csv` : Train avec nouvelles features
* `test_with_features.csv` : Test avec nouvelles features
* `feature_engineering_params.pkl` : Paramètres du pipeline

## Section 0 : Imports

In [1]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')

# Ajouter src au path
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# Import module feature_engineering
from src.feature_engineering import (
    create_ratio_features,
    create_interaction_features,
    create_temporal_features,
    create_polynomial_features,
    fit_aggregated_features,
    transform_aggregated_features,
    print_feature_summary
)


In [2]:
# Configuration
REFERENCE_YEAR = 2016  # Année du dataset

# Chemins
processed = Path('../data/processed_data')
interim = Path('../data/interim_data')
interim.mkdir(parents=True, exist_ok=True)

print(f"Année de référence: {REFERENCE_YEAR}")

Année de référence: 2016


## Section 1 : Chargement des Données Traitées

In [11]:
# Charger les données du notebook 02
train_df = pd.read_csv(processed / 'train_processed.csv')
test_df = pd.read_csv(processed / 'test_processed.csv')

print(f"Train: {train_df.shape}")
print(f"Test: {test_df.shape}")
print(f"\nColonnes: {train_df.shape[1]}")

Train: (1332, 24)
Test: (334, 24)

Colonnes: 24


In [37]:
initial_columns = train_df.columns

In [12]:
# Vérifier les colonnes disponibles
print("Colonnes disponibles:")
for col in train_df.columns[:24]:
    print(f"  - {col}")

Colonnes disponibles:
  - OSEBuildingID
  - BuildingType
  - PrimaryPropertyType
  - Address
  - Neighborhood
  - Latitude
  - Longitude
  - YearBuilt
  - NumberofBuildings
  - NumberofFloors
  - PropertyGFATotal
  - PropertyGFAParking
  - PropertyGFABuilding(s)
  - ListOfAllPropertyUseTypes
  - LargestPropertyUseType
  - LargestPropertyUseTypeGFA
  - SecondLargestPropertyUseType
  - SecondLargestPropertyUseTypeGFA
  - ENERGYSTARScore
  - SteamUse(kBtu)
  - Electricity(kWh)
  - NaturalGas(therms)
  - TotalGHGEmissions
  - TotalGHGEmissions_log


## Section 2 : Identification des Variables Disponibles

Nous identifions les variables numériques qui peuvent être utilisées pour créer des features dérivées.

In [13]:
# Variables cibles (à exclure du feature engineering)
target_vars = ['TotalGHGEmissions', 'TotalGHGEmissions_log']

# Variables numériques disponibles
numeric_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [c for c in numeric_cols if c not in target_vars]

print(f"Total : {len(numeric_cols)}")
print(f"Liste :")
for col in numeric_cols:
    print(f"  - {col}")

Total : 15
Liste :
  - OSEBuildingID
  - Latitude
  - Longitude
  - YearBuilt
  - NumberofBuildings
  - NumberofFloors
  - PropertyGFATotal
  - PropertyGFAParking
  - PropertyGFABuilding(s)
  - LargestPropertyUseTypeGFA
  - SecondLargestPropertyUseTypeGFA
  - ENERGYSTARScore
  - SteamUse(kBtu)
  - Electricity(kWh)
  - NaturalGas(therms)


## Section 3 : Features Ratios

### Définition
Les features ratios combinent deux variables par division pour créer des indicateurs d'efficacité ou de proportion.

### Exemples
- Surface par étage
- Ratio parking / surface totale

In [14]:
# Définir les ratios à créer (ajuster selon colonnes disponibles)
ratio_definitions = {}
ratio_source_vars = []  # Tracker variables sources

# Vérifier disponibilité et créer ratios
if 'PropertyGFATotal' in train_df.columns and 'NumberofFloors' in train_df.columns:
    ratio_definitions['GFA_per_floor'] = ('PropertyGFATotal', 'NumberofFloors')
    ratio_source_vars.extend(['PropertyGFATotal', 'NumberofFloors'])

if 'PropertyGFAParking' in train_df.columns and 'PropertyGFATotal' in train_df.columns:
    ratio_definitions['Parking_ratio'] = ('PropertyGFAParking', 'PropertyGFATotal')
    if 'PropertyGFAParking' not in ratio_source_vars:
        ratio_source_vars.append('PropertyGFAParking')
    if 'PropertyGFATotal' not in ratio_source_vars:
        ratio_source_vars.append('PropertyGFATotal')

if 'PropertyGFABuilding(s)' in train_df.columns and 'PropertyGFATotal' in train_df.columns:
    ratio_definitions['Building_ratio'] = ('PropertyGFABuilding(s)', 'PropertyGFATotal')
    if 'PropertyGFABuilding(s)' not in ratio_source_vars:
        ratio_source_vars.append('PropertyGFABuilding(s)')
    if 'PropertyGFATotal' not in ratio_source_vars:
        ratio_source_vars.append('PropertyGFATotal')

if 'PropertyGFATotal' in train_df.columns and 'NumberofBuildings' in train_df.columns:
    ratio_definitions['GFA_per_building'] = ('PropertyGFATotal', 'NumberofBuildings')
    if 'PropertyGFATotal' not in ratio_source_vars:
        ratio_source_vars.append('PropertyGFATotal')
    if 'NumberofBuildings' not in ratio_source_vars:
        ratio_source_vars.append('NumberofBuildings')

print(f"Total : {len(ratio_definitions)}")
for name, (num, denom) in ratio_definitions.items():
    print(f"  {name} = {num} / {denom}")

print(f"\nVariables sources à supprimer après : {len(set(ratio_source_vars))}")
print(f"  {sorted(set(ratio_source_vars))}")

Total : 4
  GFA_per_floor = PropertyGFATotal / NumberofFloors
  Parking_ratio = PropertyGFAParking / PropertyGFATotal
  Building_ratio = PropertyGFABuilding(s) / PropertyGFATotal
  GFA_per_building = PropertyGFATotal / NumberofBuildings

Variables sources à supprimer après : 5
  ['NumberofBuildings', 'NumberofFloors', 'PropertyGFABuilding(s)', 'PropertyGFAParking', 'PropertyGFATotal']


In [15]:
# Créer ratios
if len(ratio_definitions) > 0:
    train_with_ratios = create_ratio_features(train_df, ratio_definitions)
    test_with_ratios = create_ratio_features(test_df, ratio_definitions)
    print(f"Ratios créés : {len(ratio_definitions)}")
else:
    train_with_ratios = train_df.copy()
    test_with_ratios = test_df.copy()
    print("Aucun ratio créé")

Ratios créés : 4


## Section 4 : Features Temporelles

### Définition
Features basées sur le temps : âge des bâtiments, ancienneté.

### Features créées
- Building_age : Âge en années
- Building_age_squared : Âge au carré (relation non-linéaire)
- Is_old_building : Indicateur binaire (>50 ans)

In [16]:
# Créer temporelles
train_with_temporal = create_temporal_features(train_with_ratios, REFERENCE_YEAR)
test_with_temporal = create_temporal_features(test_with_ratios, REFERENCE_YEAR)

# Identifier sources à supprimer
temporal_source_vars = []
if 'Building_age' in train_with_temporal.columns:
    # On garde Building_age de base, on supprime YearBuilt
    if 'YearBuilt' in train_with_temporal.columns:
        temporal_source_vars.append('YearBuilt')
    # Si on a Building_age_squared, pas besoin de supprimer Building_age (info complémentaire)


print("="*80)
print("FEATURES TEMPORELLES")
print("="*80)

temporal_created = [c for c in ['Building_age', 'Building_age_squared', 'Is_old_building'] if c in train_with_temporal.columns]
print(f"Features créées : {len(temporal_created)}")
for feat in temporal_created:
    print(f"  - {feat}")

print(f"Variables sources à supprimer après : {len(temporal_source_vars)}")
if temporal_source_vars:
    print(f"  {temporal_source_vars}")

FEATURES TEMPORELLES
Features créées : 3
  - Building_age
  - Building_age_squared
  - Is_old_building
Variables sources à supprimer après : 1
  ['YearBuilt']


In [17]:
# Vérifier la distribution de l'âge
if 'Building_age' in train_with_temporal.columns:
    print("Distribution de l'âge des bâtiments:")
    print(train_with_temporal['Building_age'].describe())
    print(f"\nBâtiments anciens (>50 ans): {train_with_temporal['Is_old_building'].sum()}")

Distribution de l'âge des bâtiments:
count    1332.000000
mean       54.280030
std        32.386756
min         7.000000
25%        27.000000
50%        51.000000
75%        86.000000
max       110.000000
Name: Building_age, dtype: float64

Bâtiments anciens (>50 ans): 677


## Section 5 : Features Interactions

### Définition
Multiplication de deux variables pour capturer leurs effets combinés.

### Exemples
- Size × Floors : Volume approximatif
- Age × Size : Vieux grands bâtiments vs nouveaux petits

In [18]:
# Définir interactions
interaction_definitions = {}
interaction_source_vars = []

if 'PropertyGFATotal' in train_with_temporal.columns and 'NumberofFloors' in train_with_temporal.columns:
    interaction_definitions['Size_floors'] = ('PropertyGFATotal', 'NumberofFloors')
    if 'PropertyGFATotal' not in interaction_source_vars:
        interaction_source_vars.append('PropertyGFATotal')
    if 'NumberofFloors' not in interaction_source_vars:
        interaction_source_vars.append('NumberofFloors')

if 'Building_age' in train_with_temporal.columns and 'PropertyGFATotal' in train_with_temporal.columns:
    interaction_definitions['Age_size'] = ('Building_age', 'PropertyGFATotal')
    if 'Building_age' not in interaction_source_vars:
        interaction_source_vars.append('Building_age')
    if 'PropertyGFATotal' not in interaction_source_vars:
        interaction_source_vars.append('PropertyGFATotal')

if 'Building_age' in train_with_temporal.columns and 'NumberofFloors' in train_with_temporal.columns:
    interaction_definitions['Age_floors'] = ('Building_age', 'NumberofFloors')
    if 'Building_age' not in interaction_source_vars:
        interaction_source_vars.append('Building_age')
    if 'NumberofFloors' not in interaction_source_vars:
        interaction_source_vars.append('NumberofFloors')

print("="*80)
print("FEATURES INTERACTIONS")
print("="*80)
print(f"\nInteractions à créer : {len(interaction_definitions)}")
for name, (var1, var2) in interaction_definitions.items():
    print(f"  {name} = {var1} × {var2}")

print(f"\nVariables sources à supprimer après : {len(set(interaction_source_vars))}")
print(f"  {sorted(set(interaction_source_vars))}")

FEATURES INTERACTIONS

Interactions à créer : 3
  Size_floors = PropertyGFATotal × NumberofFloors
  Age_size = Building_age × PropertyGFATotal
  Age_floors = Building_age × NumberofFloors

Variables sources à supprimer après : 3
  ['Building_age', 'NumberofFloors', 'PropertyGFATotal']


In [19]:
# Créer interactions
if len(interaction_definitions) > 0:
    train_with_inter = create_interaction_features(train_with_temporal, interaction_definitions)
    test_with_inter = create_interaction_features(test_with_temporal, interaction_definitions)
    print(f"Interactions créées : {len(interaction_definitions)}")
else:
    train_with_inter = train_with_temporal.copy()
    test_with_inter = test_with_temporal.copy()
    print("Aucune interaction créée")

Interactions créées : 3


## Section 6 : Features Polynomiales

### Définition
Puissances et racines de variables pour capturer des relations non-linéaires.

### Exemples
- Surface² : Effet quadratique
- √Surface : Effet atténué

In [21]:
# Définir polynômes
polynomial_definitions = {}
polynomial_source_vars = []

if 'PropertyGFATotal' in train_with_inter.columns:
    polynomial_definitions['GFA_squared'] = ('PropertyGFATotal', 2)
    polynomial_definitions['GFA_sqrt'] = ('PropertyGFATotal', 0.5)
    if 'PropertyGFATotal' not in polynomial_source_vars:
        polynomial_source_vars.append('PropertyGFATotal')

if 'NumberofFloors' in train_with_inter.columns:
    polynomial_definitions['Floors_squared'] = ('NumberofFloors', 2)
    if 'NumberofFloors' not in polynomial_source_vars:
        polynomial_source_vars.append('NumberofFloors')

print("="*80)
print("FEATURES POLYNOMIALES")
print("="*80)
print(f"\nPolynômes à créer : {len(polynomial_definitions)}")
for name, (var, power) in polynomial_definitions.items():
    print(f"  {name} = {var}^{power}")

print(f"\nVariables sources à supprimer après : {len(set(polynomial_source_vars))}")
print(f"  {sorted(set(polynomial_source_vars))}")

FEATURES POLYNOMIALES

Polynômes à créer : 3
  GFA_squared = PropertyGFATotal^2
  GFA_sqrt = PropertyGFATotal^0.5
  Floors_squared = NumberofFloors^2

Variables sources à supprimer après : 2
  ['NumberofFloors', 'PropertyGFATotal']


In [22]:
# Créer polynômes
if len(polynomial_definitions) > 0:
    train_with_poly = create_polynomial_features(train_with_inter, polynomial_definitions)
    test_with_poly = create_polynomial_features(test_with_inter, polynomial_definitions)
    print(f"\nPolynômes créés : {len(polynomial_definitions)}")
else:
    train_with_poly = train_with_inter.copy()
    test_with_poly = test_with_inter.copy()
    print("\nAucun polynôme créé")


Polynômes créés : 3


## Section 7 : Features Agrégées (FIT/TRANSFORM)

### Définition
Statistiques par groupe (quartier, type de propriété, etc.).

### IMPORTANT
Ces features utilisent la variable cible, donc :
- **FIT** : Calculer statistiques sur TRAIN uniquement
- **TRANSFORM** : Appliquer ces statistiques sur TRAIN et TEST

### Exemples
- Émission moyenne par quartier
- Émission médiane par type de propriété

In [23]:
# Identifier colonnes de groupement
possible_groupby = ['Neighborhood', 'PrimaryPropertyType', 'ZipCode']
available_groupby = [col for col in possible_groupby if col in train_with_poly.columns]

print("="*80)
print("FEATURES AGRÉGÉES")
print("="*80)
print(f"\nColonnes de groupement disponibles : {len(available_groupby)}")
for col in available_groupby:
    print(f"  - {col}")

# Note : On GARDE les variables de groupement (info différente des agrégations)
print(f"\nCes colonnes seront CONSERVÉES (info complémentaire aux agrégations)")

FEATURES AGRÉGÉES

Colonnes de groupement disponibles : 2
  - Neighborhood
  - PrimaryPropertyType

Ces colonnes seront CONSERVÉES (info complémentaire aux agrégations)


In [24]:
# FIT et TRANSFORM
if len(available_groupby) > 0 and 'TotalGHGEmissions_log' in train_with_poly.columns:
    agg_params = fit_aggregated_features(
        train_with_poly,
        target_col='TotalGHGEmissions_log',
        groupby_cols=available_groupby,
        agg_functions=['mean', 'median', 'std']
    )
    
    train_with_agg = transform_aggregated_features(train_with_poly, agg_params)
    test_with_agg = transform_aggregated_features(test_with_poly, agg_params)
    
    n_agg = len(available_groupby) * len(agg_params['agg_functions'])
    print(f"\nFeatures agrégées créées : {n_agg}")
else:
    agg_params = None
    train_with_agg = train_with_poly.copy()
    test_with_agg = test_with_poly.copy()
    print("\nAucune feature agrégée créée")


Features agrégées créées : 6


## Section 7 : Suppression Variables Sources

### Principe
Supprimer les variables sources des features dérivées pour éviter multicolinéarité.

### Règles appliquées
- Ratios : Supprimer les 2 sources
- Interactions : Supprimer les 2 sources
- Polynômes : Supprimer la source
- Temporelles : Supprimer YearBuilt (garder Building_age)
- Agrégées : Garder les variables de groupement

In [25]:
# Consolider toutes les variables sources à supprimer
all_source_vars = set()
all_source_vars.update(ratio_source_vars)
all_source_vars.update(temporal_source_vars)
all_source_vars.update(interaction_source_vars)
all_source_vars.update(polynomial_source_vars)

# Filtrer celles qui existent dans le dataset
vars_to_drop = [v for v in all_source_vars if v in train_with_agg.columns]

print("="*80)
print("SUPPRESSION VARIABLES SOURCES")
print("="*80)
print(f"\nVariables sources identifiées : {len(all_source_vars)}")
print(f"Variables à supprimer : {len(vars_to_drop)}")
print(f"\nListe :")
for var in sorted(vars_to_drop):
    print(f"  - {var}")

SUPPRESSION VARIABLES SOURCES

Variables sources identifiées : 7
Variables à supprimer : 7

Liste :
  - Building_age
  - NumberofBuildings
  - NumberofFloors
  - PropertyGFABuilding(s)
  - PropertyGFAParking
  - PropertyGFATotal
  - YearBuilt


In [26]:
# Supprimer
if len(vars_to_drop) > 0:
    train_clean = train_with_agg.drop(columns=vars_to_drop)
    test_clean = test_with_agg.drop(columns=vars_to_drop)
    
    print(f"\nSuppression effectuée")
    print(f"  Avant : {train_with_agg.shape}")
    print(f"  Après : {train_clean.shape}")
    print(f"  Colonnes supprimées : {len(vars_to_drop)}")
else:
    train_clean = train_with_agg.copy()
    test_clean = test_with_agg.copy()
    print(f"\nAucune variable à supprimer")


Suppression effectuée
  Avant : (1332, 43)
  Après : (1332, 36)
  Colonnes supprimées : 7


## Section 8 : Vérification Multicolinéarité

In [27]:
# Calculer matrice de corrélation sur variables numériques
numeric_cols = train_clean.select_dtypes(include=[np.number]).columns
target_vars = ['TotalGHGEmissions', 'TotalGHGEmissions_log']
numeric_features = [c for c in numeric_cols if c not in target_vars]

if len(numeric_features) > 0:
    corr_matrix = train_clean[numeric_features].corr().abs()
    
    # Identifier paires haute corrélation (>0.9)
    high_corr_pairs = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            if corr_matrix.iloc[i, j] > 0.9:
                high_corr_pairs.append((
                    corr_matrix.columns[i],
                    corr_matrix.columns[j],
                    corr_matrix.iloc[i, j]
                ))
    
    print("="*80)
    print("VÉRIFICATION MULTICOLINÉARITÉ")
    print("="*80)
    print(f"\nPaires avec corrélation > 0.9 : {len(high_corr_pairs)}")
    
    if len(high_corr_pairs) > 0:
        print(f"\nATTENTION : Corrélations élevées détectées")
        for var1, var2, corr in high_corr_pairs[:10]:  # Afficher max 10
            print(f"  {var1} <-> {var2} : {corr:.3f}")
        if len(high_corr_pairs) > 10:
            print(f"  ... et {len(high_corr_pairs) - 10} autres paires")
    else:
        print(f"\nOK : Pas de corrélation excessive (>0.9)")
else:
    print("Pas de variables numériques à vérifier")

VÉRIFICATION MULTICOLINÉARITÉ

Paires avec corrélation > 0.9 : 9

ATTENTION : Corrélations élevées détectées
  LargestPropertyUseTypeGFA <-> GFA_per_building : 0.954
  LargestPropertyUseTypeGFA <-> GFA_squared : 0.927
  LargestPropertyUseTypeGFA <-> GFA_sqrt : 0.938
  Parking_ratio <-> Building_ratio : 0.939
  GFA_per_building <-> GFA_squared : 0.967
  GFA_per_building <-> GFA_sqrt : 0.983
  GFA_squared <-> GFA_sqrt : 0.906
  Neighborhood_mean <-> Neighborhood_median : 0.978
  PrimaryPropertyType_mean <-> PrimaryPropertyType_median : 0.989


In [28]:
# Section 8 : Suppression des variables redondantes
print("="*80)
print("SUPPRESSION VARIABLES REDONDANTES (MULTICOLINÉARITÉ)")
print("="*80)

vars_to_drop_multicollinearity = []

# 1. Garder seulement GFA_sqrt (éliminer GFA_squared et GFA_per_building)
if 'GFA_squared' in train_clean.columns:
    vars_to_drop_multicollinearity.append('GFA_squared')
    print(" GFA_squared (redondant avec GFA_sqrt)")

if 'GFA_per_building' in train_clean.columns:
    vars_to_drop_multicollinearity.append('GFA_per_building')
    print(" GFA_per_building (redondant avec GFA_sqrt)")

# 2. Garder seulement Parking_ratio (éliminer Building_ratio)
if 'Building_ratio' in train_clean.columns:
    vars_to_drop_multicollinearity.append('Building_ratio')
    print(" Building_ratio (redondant avec Parking_ratio)")

# 3. Garder seulement mean pour les agrégations (éliminer median)
for col in train_clean.columns:
    if '_median' in col:
        vars_to_drop_multicollinearity.append(col)
        print(f" {col} (redondant avec _mean)")



print(f"\nTotal variables à supprimer : {len(vars_to_drop_multicollinearity)}")

# Appliquer la suppression
train_clean = train_clean.drop(columns=vars_to_drop_multicollinearity)
test_clean = test_clean.drop(columns=vars_to_drop_multicollinearity)

print(f"\nAprès nettoyage multicolinéarité :")
print(f"  Train : {train_clean.shape}")
print(f"  Test : {test_clean.shape}")

SUPPRESSION VARIABLES REDONDANTES (MULTICOLINÉARITÉ)
 GFA_squared (redondant avec GFA_sqrt)
 GFA_per_building (redondant avec GFA_sqrt)
 Building_ratio (redondant avec Parking_ratio)
 Neighborhood_median (redondant avec _mean)
 PrimaryPropertyType_median (redondant avec _mean)

Total variables à supprimer : 5

Après nettoyage multicolinéarité :
  Train : (1332, 31)
  Test : (334, 31)


In [ ]:
# Vérifier qu'il n'y a plus de corrélations > 0.9
numeric_cols = train_clean.select_dtypes(include=[np.number]).columns
target_vars = ['TotalGHGEmissions', 'TotalGHGEmissions_log']
numeric_features = [c for c in numeric_cols if c not in target_vars]

corr_matrix = train_clean[numeric_features].corr().abs()

high_corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if corr_matrix.iloc[i, j] > 0.9:
            high_corr_pairs.append((
                corr_matrix.columns[i],
                corr_matrix.columns[j],
                corr_matrix.iloc[i, j]
            ))

print("\n" + "="*80)
print("VÉRIFICATION POST-NETTOYAGE")
print("="*80)
print(f"Paires avec corrélation > 0.9 : {len(high_corr_pairs)}")

if len(high_corr_pairs) > 0:
    for var1, var2, corr in high_corr_pairs:
        print(f"  {var1} <-> {var2} : {corr:.3f}")
else:
    print("cOK : Pas de multicolinéarité excessive")


VÉRIFICATION POST-NETTOYAGE
Paires avec corrélation > 0.9 : 1
  LargestPropertyUseTypeGFA <-> GFA_sqrt : 0.938


## Section 9 : Nettoyage Final

In [30]:
# Traiter NA et Inf créés
na_train = train_clean.isnull().sum().sum()
na_test = test_clean.isnull().sum().sum()
inf_train = np.isinf(train_clean.select_dtypes(include=[np.number])).sum().sum()
inf_test = np.isinf(test_clean.select_dtypes(include=[np.number])).sum().sum()

print("="*80)
print("NETTOYAGE FINAL")
print("="*80)
print(f"\nAvant nettoyage :")
print(f"  NA train : {na_train}")
print(f"  NA test : {na_test}")
print(f"  Inf train : {inf_train}")
print(f"  Inf test : {inf_test}")

if na_train > 0 or na_test > 0 or inf_train > 0 or inf_test > 0:
    # Identifier nouvelles colonnes
    new_cols = [c for c in train_clean.columns if c not in initial_columns]
    
    # Remplacer Inf par NaN puis NaN par 0
    for col in new_cols:
        if col in train_clean.columns:
            train_clean[col] = train_clean[col].replace([np.inf, -np.inf], np.nan).fillna(0)
            test_clean[col] = test_clean[col].replace([np.inf, -np.inf], np.nan).fillna(0)
    
    print(f"\nAprès nettoyage :")
    print(f"  NA train : {train_clean.isnull().sum().sum()}")
    print(f"  NA test : {test_clean.isnull().sum().sum()}")
    print(f"  Inf train : {np.isinf(train_clean.select_dtypes(include=[np.number])).sum().sum()}")
    print(f"  Inf test : {np.isinf(test_clean.select_dtypes(include=[np.number])).sum().sum()}")
else:
    print(f"\nAucun nettoyage nécessaire")

NETTOYAGE FINAL

Avant nettoyage :
  NA train : 0
  NA test : 0
  Inf train : 0
  Inf test : 0

Aucun nettoyage nécessaire


## Section 10 : Résumé et Vérifications

In [32]:
print("="*80)
print("RÉSUMÉ FEATURE ENGINEERING")
print("="*80)

# Détail
n_created = len(ratio_definitions) + len(temporal_created) + len(interaction_definitions) + len(polynomial_definitions)
if agg_params:
    n_created += len(available_groupby) * len(agg_params['agg_functions'])

print(f"\nFeatures créées : {n_created}")
print(f"  Ratios : {len(ratio_definitions)}")
print(f"  Temporelles : {len(temporal_created)}")
print(f"  Interactions : {len(interaction_definitions)}")
print(f"  Polynomiales : {len(polynomial_definitions)}")
if agg_params:
    print(f"  Agrégées : {len(available_groupby) * len(agg_params['agg_functions'])}")

print(f"\nVariables sources supprimées : {len(vars_to_drop)}")

print(f"\nBilan : +{n_created} créées, -{len(vars_to_drop)} supprimées = {n_created - len(vars_to_drop)} nettes")

RÉSUMÉ FEATURE ENGINEERING

Features créées : 19
  Ratios : 4
  Temporelles : 3
  Interactions : 3
  Polynomiales : 3
  Agrégées : 6

Variables sources supprimées : 7

Bilan : +19 créées, -7 supprimées = 12 nettes


In [33]:
# Vérifications finales
print("\n" + "="*80)
print("VÉRIFICATIONS FINALES")
print("="*80)

check1 = train_clean.shape[1] == test_clean.shape[1]
check2 = train_clean.isnull().sum().sum() == 0
check3 = test_clean.isnull().sum().sum() == 0
check4 = np.isinf(train_clean.select_dtypes(include=[np.number])).sum().sum() == 0
check5 = np.isinf(test_clean.select_dtypes(include=[np.number])).sum().sum() == 0

print(f"\n1. Même nb colonnes train/test : {check1}")
print(f"2. Pas de NA train : {check2}")
print(f"3. Pas de NA test : {check3}")
print(f"4. Pas de Inf train : {check4}")
print(f"5. Pas de Inf test : {check5}")

all_ok = all([check1, check2, check3, check4, check5])
print(f"\nStatut global : {'OK' if all_ok else 'PROBLÈME DÉTECTÉ'}")


VÉRIFICATIONS FINALES

1. Même nb colonnes train/test : True
2. Pas de NA train : True
3. Pas de NA test : True
4. Pas de Inf train : True
5. Pas de Inf test : True

Statut global : OK


## Section 11 : Sauvegarde

In [35]:
# Sauvegarder datasets
train_clean.to_csv(interim / 'train_with_features.csv', index=False)
test_clean.to_csv(interim / 'test_with_features.csv', index=False)

print(f"\ntrain_with_features.csv : {train_clean.shape}")
print(f"test_with_features.csv : {test_clean.shape}")
print(f"\nLocalisation : {interim}")


train_with_features.csv : (1332, 31)
test_with_features.csv : (334, 31)

Localisation : ..\data\interim_data


In [38]:
# Sauvegarder paramètres
feature_params = {
    'ratio_definitions': ratio_definitions,
    'interaction_definitions': interaction_definitions,
    'polynomial_definitions': polynomial_definitions,
    'aggregated_features': agg_params if agg_params else {},
    'reference_year': REFERENCE_YEAR,
    'n_features_initial': len(initial_columns),
    'n_features_final': len(train_clean.columns),
    'n_features_created': n_created,
    'n_features_dropped': len(vars_to_drop),
    'dropped_source_vars': sorted(vars_to_drop),
    'high_corr_pairs': len(high_corr_pairs) if 'high_corr_pairs' in locals() else 0
}

joblib.dump(feature_params, interim / 'feature_engineering_params.pkl')

print("\nPARAMÈTRES SAUVEGARDÉS")
print(f"feature_engineering_params.pkl")


PARAMÈTRES SAUVEGARDÉS
feature_engineering_params.pkl


In [39]:
print("\n" + "="*80)
print("FEATURE ENGINEERING TERMINÉ")
print("="*80)
print(f"Prochaine étape : Feature Selection (Notebook 04)")



FEATURE ENGINEERING TERMINÉ
Prochaine étape : Feature Selection (Notebook 04)
